# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment,Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

import azureml.core
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

import os
import joblib
import pandas as pd
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace

Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-132641
aml-quickstarts-132641
southcentralus
cdbe0b43-92a0-4715-838a-f2648cc7ad21


## Create an Azure ML experiment

In [3]:
experiment_name = 'automl-experiment'
project_folder = './automl-experiment'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment,quick-starts-ws-132641,Link to Azure Machine Learning studio,Link to Documentation


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
dataset=Dataset.get_by_name(ws,name="Credit-Card-Churners")
run = experiment.start_logging()
df = dataset.to_pandas_dataframe()
df.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
count,1.012700e+04,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,7.391776e+08,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894,0.159997,0.840003
std,3.690378e+07,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691,0.365301,0.365301
min,7.080821e+08,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000,0.000008,0.000420
25%,7.130368e+08,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000,0.000099,0.999660
50%,7.179264e+08,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000,0.000181,0.999820
75%,7.731435e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000,0.000337,0.999900
max,8.283431e+08,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000,0.999580,0.999990


In [5]:
currDir=os.getcwd()
print(currDir)
os.listdir(currDir)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-aml-cluster/code/Users/odl_user_132641


['.azureml',
 '.config',
 '.ipynb_checkpoints',
 'automl(1).ipynb',
 'automl-experiment',
 'automl.ipynb',
 'automl.log',
 'automltry.ipynb',
 'automl_errors.log',
 'azureml_automl.log',
 'conda_dependencies.yml',
 'CreditCardChurners.csv',
 'hyperparameter_tuning new ScriptRunConfig.ipynb',
 'hyperparameter_tuning.ipynb',
 'outputs',
 'README.md',
 'train.py',
 'training',
 'udacity-project.ipynb',
 '__pycache__']

## Config Compute Cluster

In [6]:
compute_name = "compute-aml-cluster"

# Check if the compute target exists
try:
    compute_aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisionibng_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=5)
    compute_aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

compute_aml_cluster.wait_for_completion(show_output=True)

Found existing cluster.

Running


In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Use the clean_data function to clean your data.
#x, y = clean_data(df)

def pre_processing(data):
    # Clean and one hot encode data
    x_df = data.to_pandas_dataframe().dropna()
    x_df.drop("CLIENTNUM", inplace=True, axis=1)
    gender = pd.get_dummies(x_df.Gender, prefix="Gender")
    x_df.drop("Gender", inplace=True, axis=1)
    education_level = pd.get_dummies(x_df.Education_Level, prefix="Education_Level")
    x_df.drop("Education_Level", inplace=True, axis=1)
    x_df = x_df.join(education_level)
    marital_status = pd.get_dummies(x_df.Marital_Status, prefix="Marital_Status")
    x_df.drop("Marital_Status", inplace=True, axis=1)
    x_df = x_df.join(marital_status)
    income_category = pd.get_dummies(x_df.Income_Category, prefix="Income_Category")
    x_df.drop("Income_Category", inplace=True, axis=1)
    x_df = x_df.join(income_category)
    card_category = pd.get_dummies(x_df.Card_Category, prefix="Card_Category")
    x_df.drop("Card_Category", inplace=True, axis=1)
    x_df = x_df.join(card_category)
    y_df = x_df.pop("Attrition_Flag").apply(lambda s: 1 if s == "Attrited Customer" else 0)

    return x_df, y_df

x, y = pre_processing(dataset)

all_data = pd.concat([x,y], axis = 1)
all_data.head()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,...,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver,Attrition_Flag
0,45,3,39,5,1,3,12691.0,777,11914.0,1.335,...,0,1,0,0,0,1,0,0,0,0
1,49,5,44,6,1,2,8256.0,864,7392.0,1.541,...,0,0,0,1,0,1,0,0,0,0
2,51,3,36,4,1,0,3418.0,0,3418.0,2.594,...,0,0,1,0,0,1,0,0,0,0
3,40,4,34,3,4,1,3313.0,2517,796.0,1.405,...,0,0,0,1,0,1,0,0,0,0
4,40,3,21,5,1,0,4716.0,0,4716.0,2.175,...,0,1,0,0,0,1,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(all_data, test_size=0.2,random_state=0)
# columns=x_train.columns
print(x_train.head(3))
print(x_test.head(3))
print(x_train.shape)
print(x_test.shape)

if "training" not in os.listdir():
    os.mkdir("./training")

x_train.to_csv('training/train_data.csv')
x_test.to_csv('training/test_data.csv')

# x_train.to_csv(path_or_buf='training/train_data.csv', columns=columns, header=True, index=False) 
# x_test.to_csv(path_or_buf='training/test_data.csv', columns=columns, header=True, index=False)

      Customer_Age  Dependent_count  Months_on_book  Total_Relationship_Count  \
147             55                2              50                         3   
1985            32                0              27                         6   
8316            35                4              25                         2   

      Months_Inactive_12_mon  Contacts_Count_12_mon  Credit_Limit  \
147                        2                      0       10916.0   
1985                       3                      3        9238.0   
8316                       3                      2        3379.0   

      Total_Revolving_Bal  Avg_Open_To_Buy  Total_Amt_Chng_Q4_Q1  ...  \
147                  1536           9380.0                 1.317  ...   
1985                    0           9238.0                 0.809  ...   
8316                  587           2792.0                 0.670  ...   

      Income_Category_$40K - $60K  Income_Category_$60K - $80K  \
147                             0      

In [ ]:
# get the datastore to upload prepared data
datastore = ws.get_default_datastore()
print(datastore)

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='training/', target_path='data/')
# datastore.upload_files(['trainset.csv'])

# create a dataset referencing the cloud location
train_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ('data/train_data.csv'))])

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-c4e12a72-947f-49da-baaf-313d089b8c46",
  "account_name": "mlstrg132641",
  "protocol": "https",
  "endpoint": "core.windows.net"
}
Uploading an estimated of 2 files
Target already exists. Skipping upload for data/test_data.csv
Target already exists. Skipping upload for data/train_data.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
remote_runl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 3,
    "max_cores_per_iteration":-1
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_aml_cluster,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="Attrition_Flag",   
                             path = project_folder,
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log"
)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
remote_run

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_automl_best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()
print('Best Run ID',best_run.id)


In [ ]:
best_run

In [ ]:
fitted_automl_best_model

In [ ]:
# Metric in best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print('\nAccuracy of Best run',best_run_metrics['accuracy'],sep='\n')
print(best_run)

model_name = best_run.properties['model_name']
model_name

In [ ]:
#TODO: Save the best model

os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service